In [2]:
import requests
from requests import Session
from requests.exceptions import HTTPError
import pprint
import json
import time
import pandas as pd
from random import uniform

In [3]:
pp = pprint.PrettyPrinter(indent=4)

In [21]:
try:
    response = requests.get('https://statsapi.mlb.com/api/v1/teams?sportId=1')
    response.raise_for_status()
    
    # access JSON content
    jsonResponse = response.json()
    #for k,v in jsonResponse.items():
        #print(k)
    #pp.pprint(jsonResponse)
    #pp.pprint(jsonResponse["playsByInning"])
    #pp.pprint(jsonResponse["allPlays"][0])
    #pp.pprint(jsonResponse["allPlays"][0]["playEvents"][3])
    
    
    for teams in jsonResponse["teams"]:
        pp.pprint(f"{teams['name']} - {teams['id']}")
    #    if plays["isPitch"] == True:
    #        pp.pprint(plays['count'])
    #        pp.pprint(plays['pitchNumber'])
    #        pp.pprint(plays['details'])
    #        pp.pprint(plays['pitchData'])

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

'Los Angeles Angels - 108'
'Arizona Diamondbacks - 109'
'Baltimore Orioles - 110'
'Boston Red Sox - 111'
'Chicago Cubs - 112'
'Cincinnati Reds - 113'
'Cleveland Indians - 114'
'Colorado Rockies - 115'
'Detroit Tigers - 116'
'Houston Astros - 117'
'Kansas City Royals - 118'
'Los Angeles Dodgers - 119'
'Washington Nationals - 120'
'New York Mets - 121'
'Oakland Athletics - 133'
'Pittsburgh Pirates - 134'
'San Diego Padres - 135'
'Seattle Mariners - 136'
'San Francisco Giants - 137'
'St. Louis Cardinals - 138'
'Tampa Bay Rays - 139'
'Texas Rangers - 140'
'Toronto Blue Jays - 141'
'Minnesota Twins - 142'
'Philadelphia Phillies - 143'
'Atlanta Braves - 144'
'Chicago White Sox - 145'
'Miami Marlins - 146'
'New York Yankees - 147'
'Milwaukee Brewers - 158'


In [22]:
try:
    response = requests.get('https://statsapi.mlb.com/api/v1/game/565896/playByPlay')
    response.raise_for_status()
    
    # access JSON content
    jsonResponse = response.json()
    pp.pprint(jsonResponse["allPlays"][1])


except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

{   'about': {   'atBatIndex': 1,
                 'captivatingIndex': 33,
                 'endTime': '2019-03-30T17:07:53.215Z',
                 'halfInning': 'top',
                 'hasOut': False,
                 'hasReview': False,
                 'inning': 1,
                 'isComplete': True,
                 'isScoringPlay': False,
                 'isTopInning': True,
                 'startTime': '2019-03-30T17:06:31.701Z'},
    'actionIndex': [],
    'atBatIndex': 1,
    'count': {'balls': 1, 'outs': 1, 'strikes': 1},
    'matchup': {   'batSide': {'code': 'R', 'description': 'Right'},
                   'batter': {   'fullName': 'Pete Alonso',
                                 'id': 624413,
                                 'link': '/api/v1/people/624413'},
                   'batterHotColdZones': [],
                   'pitchHand': {'code': 'R', 'description': 'Right'},
                   'pitcher': {   'fullName': 'Stephen Strasburg',
                                 

In [23]:
try:
    response = requests.get('https://statsapi.mlb.com/api/v1/seasons?sportId=1')
    response.raise_for_status()
    
    # access JSON content
    jsonResponse = response.json()
    pp.pprint(jsonResponse)


except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')



{   'copyright': 'Copyright 2020 MLB Advanced Media, L.P.  Use of any content '
                 'on this page acknowledges agreement to the terms posted here '
                 'http://gdx.mlb.com/components/copyright.txt',
    'seasons': [   {   'hasWildcard': True,
                       'postSeasonEndDate': '2020-10-28',
                       'postSeasonStartDate': '2020-09-29',
                       'preSeasonEndDate': '2020-03-24',
                       'preSeasonStartDate': '2020-02-21',
                       'regularSeasonEndDate': '2020-09-27',
                       'regularSeasonStartDate': '2020-03-26',
                       'seasonEndDate': '2020-10-28',
                       'seasonId': '2020',
                       'seasonStartDate': '2020-02-21'}]}


In [24]:
seasonStart = '2019-03-20'
seasonEnd = '2019-09-29'
seasonScheduleResponse = requests.get(f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&startDate={seasonStart}&endDate={seasonEnd}')
seasonScheduleResponse.raise_for_status()

# access JSON content
jsonSeasonScheduleResponse = seasonScheduleResponse.json()
#pp.pprint(jsonSeasonScheduleResponse)

gameList = []
for date in jsonSeasonScheduleResponse['dates']:
    games = date['games']
    
    for game in games:
        if game['gameType'] == 'R':
            gameList.append(game['gamePk'])

    
print(len(gameList))
            

2472


In [28]:
# Get pertinent seasons

list_of_plays = []
seasonResponse = requests.get('https://statsapi.mlb.com/api/v1/seasons/all?sportId=1')
seasonResponse.raise_for_status()

# access JSON content
jsonSeasonResponse = seasonResponse.json()
#pp.pprint(jsonResponse)

for season in jsonSeasonResponse['seasons']:
    if int(season['seasonId']) >= 2019 and int(season['seasonId']) < 2020:
        seasonStart = season['regularSeasonStartDate']
        seasonEnd = season['regularSeasonEndDate']
    else:
        continue

    print(season['seasonId'])
        
        


2019


In [31]:
# Create a class to retry failed requests
class ResilientSession(Session):
    """
    This class is supposed to retry requests that return temporary errors.
    At this moment it supports: 500, 502, 503, 504
    """

    def request(self, method, url, **kwargs):
        counter = 0

        while True:
            counter += 1

            r = super(ResilientSession, self).request(method, url, **kwargs)

            if r.status_code in [ 500, 502, 503, 504 ]:
                delay = 10 * counter
                logging.warn("Got recoverable error [%s] from %s %s, retry #%s in %ss" % (r.status_code, method, url, counter, delay))
                time.sleep(delay)
                continue

            return r

In [32]:
# Create session object
s = ResilientSession()

In [51]:
# Get pertinent seasons
try:
    list_of_plays = []
    seasonResponse = s.get('https://statsapi.mlb.com/api/v1/seasons/all?sportId=1')
    seasonResponse.raise_for_status()
    
    # access JSON content
    jsonSeasonResponse = seasonResponse.json()
    #pp.pprint(jsonResponse)
    
    for season in jsonSeasonResponse['seasons']:
        #if int(season['seasonId']) >= 2012 and int(season['seasonId']) < 2020:
        if int(season['seasonId']) >= 2018 and int(season['seasonId']) < 2019:
            seasonStart = season['regularSeasonStartDate']
            seasonEnd = season['regularSeasonEndDate']
        else:
            continue
        
        print(season['seasonId'])
        
        seasonScheduleResponse = s.get(f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&startDate={seasonStart}&endDate={seasonEnd}')
        seasonScheduleResponse.raise_for_status()

        # access JSON content
        jsonSeasonScheduleResponse = seasonScheduleResponse.json()
        #pp.pprint(jsonSeasonScheduleResponse)

        gameList = []
        for date in jsonSeasonScheduleResponse['dates']:
            games = date['games']

            for game in games:
                if game['gameType'] == 'R':
                    gameResponse = s.get(f"https://statsapi.mlb.com/api/v1/game/{game['gamePk']}/playByPlay")
                    gameResponse.raise_for_status()

                    # access JSON content
                    jsonGameResponse = gameResponse.json()
                    #pp.pprint(jsonResponse["allPlays"][0]["playEvents"])

                    for plays in jsonGameResponse["allPlays"]:
                        for event in plays["playEvents"]:
                            if event.get("isPitch") == True and event.get('details').get('description') != 'Automatic Ball':
                                if event.get('details').get('type'):
                                    pitchType = event.get('details').get('type').get('description')
                                else:
                                    pitchType = 'unknown'
                                    
                                try:
                                    list_of_plays.append({'season':season.get('seasonId'),
                                                 'game':game.get('gamePk'),
                                                 'batter':plays["matchup"]["batter"].get("id"),
                                                 'batSide':plays["matchup"]["batSide"].get("code"),
                                                 'batter':plays["matchup"]["pitcher"].get("id"),
                                                 'batSide':plays["matchup"]["pitchHand"].get("code"),
                                                 'count_balls': event.get('count').get('balls'),
                                                 'count_strikes': event.get('count').get('strikes'),
                                                 'pitch_result': event['details'].get('description'),
                                                 #'pitch_type': event['details']['type'].get('description'),
                                                 'pitch_type': pitchType,
                                                 'break_angle': event['pitchData']['breaks'].get('breakAngle'),
                                                 'break_length': event['pitchData']['breaks'].get('breakLength'),
                                                 'spin_direction': event['pitchData']['breaks'].get('spinDirection'),
                                                 'spin_rate': event['pitchData']['breaks'].get('spinRate'),
                                                 'end_speed': event['pitchData'].get('endSpeed'),
                                                 'nasty_factor': event['pitchData'].get('nastyFactor'),
                                                 'extension': event['pitchData'].get('extension'),
                                                 'plate_time': event['pitchData'].get('plateTime'),
                                                 'start_speed': event['pitchData'].get('startSpeed'),
                                                 'pitch_number': event.get('pitchNumber')
                                                })
                                except KeyError as key:
                                    print(f"Key error on {key} in game {game.get('gamePk')}")
                                    #print(f"{event}")
                                    #raise
                                    
                                
                                if len(list_of_plays) in (70000,150000,300000,550000,700000):
                                    print(f'Current length: {len(list_of_plays)}')
            time.sleep(uniform(1,3))
        print(f'Total length: {len(list_of_plays)}')
                                    
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')
    raise


2018
Current length: 70000
Current length: 150000
Current length: 300000
Current length: 550000
Current length: 700000
Total length: 738219


In [ ]:
df = pd.DataFrame(list_of_plays)

In [ ]:
df

In [ ]:
df.to_csv('pitch_data_2018.csv')

In [4]:
df = pd.read_csv('pitch_data_2018.csv')

In [5]:
df

,rowid,season,game,batter,batSide,count_balls,count_strikes,pitch_result,pitch_type,break_angle,break_length,spin_direction,spin_rate,end_speed,extension,plate_time,start_speed,pitch_number
0,0,2018,529407,544931,R,0,1,C,FF,36.0,4.8,220,2082.0,86.0,6.85,0.40,93.5,1
1,1,2018,529407,544931,R,0,2,S,FF,37.2,3.6,212,2171.0,86.3,6.62,0.40,94.1,2
2,2,2018,529407,544931,R,1,2,B,FF,38.4,3.6,214,2194.0,86.1,6.65,0.40,94.0,3
3,3,2018,529407,544931,R,1,3,S,CH,30.0,8.4,255,1873.0,81.7,7.13,0.42,88.2,4
4,4,2018,529407,544931,R,0,1,C,CU,15.6,12.0,63,2716.0,74.4,6.57,0.47,80.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917698,917698,2018,570335,607625,R,2,2,S,SL,6.0,6.0,156,2388.0,80.2,5.93,0.44,86.4,4
917699,917699,2018,570335,607625,R,2,2,F,CU,10.8,14.4,33,3305.0,72.9,5.76,0.48,79.8,5
917700,917700,2018,570335,607625,R,2,2,F,FF,22.8,3.6,199,2279.0,85.5,5.98,0.40,93.9,6
917701,917701,2018,570335,607625,R,2,2,E,SL,4.8,7.2,144,2455.0,82.0,5.61,0.43,88.3,7
